## Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.


## Import packages

In [2]:
# import packages
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib qt


# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

## Camera calibration

In [3]:
# helper funtions for camera calibration
def search_corners(nx,ny): ### tested
    # searches cornerspoints on cheesboard and returns image- and object points
    # define chessboard parameters
    #nx = 9
    #ny = 6

    # prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
    objp = np.zeros((nx*ny,3), np.float32)
    objp[:,:2] = np.mgrid[0:nx,0:ny].T.reshape(-1,2)

    # Arrays to store object points and image points from all the images.
    objpoints = [] # 3d points in real world space
    imgpoints = [] # 2d points in image plane.

    # Make a list of calibration images
    images = glob.glob('./camera_cal/calibration*.jpg')

    # Step through the list and search for chessboard corners
    for fname in images:
        img = cv2.imread(fname)
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

        # Find the chessboard corners
        ret, corners = cv2.findChessboardCorners(gray, (nx,ny),None)

        # If found, add object points, image points
        if ret == True:
            objpoints.append(objp)
            imgpoints.append(corners)

            # Draw and display the corners
            img = cv2.drawChessboardCorners(img, (9,6), corners, ret)          
    return objpoints, imgpoints


def cal_undistort(img_shape, objpoints, imgpoints):
    # undistorte images based on imgpoints and objectpoints 
    
    # calibrate camera based on imgpoints and objectpoints
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img_shape[1:], None, None)
    
    
    return mtx, dist

def undist_img(img, mtx, dist):
    # undistort image
    undist = cv2.undistort(img, mtx, dist, None, mtx)
    return undist
        

In [ ]:
# test camerea calibration
nx = 9
ny = 6

objpoints, imgpoints = search_corners(nx,ny)

# Make a list of calibration images
images = glob.glob('./camera_cal/calibration*.jpg')

for fname in images:
        img = cv2.imread(fname)
        undist, mtx, dist = cal_undistort(img, objpoints, imgpoints)
        # mtx & dist is always the same - camera specific
        mpimg.imsave("camera_cal_out/undist_"+fname[13:]+"",img)

## Create thresholded binary image

In [4]:
# helper functions
def convert_to_bin(img,sobel_thresh_min = 20, sobel_thresh_max = 100, s_thresh_min = 170, s_thresh_max = 255):
    """
    """
    
    # Convert to HLS color space and separate the S channel
    # Note: img is the undistorted image
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    s_channel = hls[:,:,2]

    # Grayscale image
    # NOTE: we already saw that standard grayscaling lost color information for the lane lines
    # Explore gradients in other colors spaces / color channels to see what might work better
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

    # Sobel x
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0) # Take the derivative in x
    abs_sobelx = np.absolute(sobelx) # Absolute x derivative to accentuate lines away from horizontal
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))

    # Threshold x gradient
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= sobel_thresh_min) & (scaled_sobel <= sobel_thresh_max)] = 1

    # Threshold color channel
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh_min) & (s_channel <= s_thresh_max)] = 1

    # Stack each channel to view their individual contributions in green and blue respectively
    # This returns a stack of the two binary images, whose components you can see as different colors
    color_binary = np.dstack(( np.zeros_like(sxbinary), sxbinary, s_binary)) * 255

    # Combine the two binary thresholds
    combined_binary = np.zeros_like(sxbinary)
    combined_binary[(s_binary == 1) | (sxbinary == 1)] = 1
    
    return combined_binary, combined_binary

    

## Perspective transformation to bird view

In [5]:
# helper function
def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    `vertices` should be a numpy array of integer points.
    """
    
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def convert_to_birdview(img, src, dst):
    # img = source image
    # src = points on source image
    # dst = points on destination image
    
    # d) use cv2.getPerspectiveTransform() to get M, the transform matrix
    M = cv2.getPerspectiveTransform(src, dst)

    # e) use cv2.warpPerspective() to warp your image to a top-down view
    #warped = np.flipud(cv2.warpPerspective(img, M, img.shape[1::-1], flags=cv2.INTER_LINEAR))
    warped = cv2.warpPerspective(img, M, img.shape[1::-1], flags=cv2.INTER_LINEAR)

    return warped, M

## Detecting Lane Lines

In [6]:
# helper functions
def hist(img):
    """generates histogram based on a binary image"""
    
    # Grab only the bottom half of the image
    # Lane lines are likely to be mostly vertical nearest to the car
    bottom_half = img[img.shape[0]//2:,:]

    # Sum across image pixels vertically - make sure to set an `axis`
    # i.e. the highest areas of vertical lines should be larger values
    histogram = np.sum(bottom_half, axis=0)
    
    return histogram

def find_lane_pixels(binary_warped):
    """finding lane line pixels using sliding window method"""
    
    # Take a histogram of the bottom half of the image
    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
    # Create an output image to draw on and visualize the result
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]//2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # HYPERPARAMETERS
    # Choose the number of sliding windows
    nwindows = 9
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 50

    # Set height of windows - based on nwindows above and image shape
    window_height = np.int(binary_warped.shape[0]//nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated later for each window in nwindows
    leftx_current = leftx_base
    rightx_current = rightx_base

    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        
        # Draw the windows on the visualization image
        cv2.rectangle(out_img,(win_xleft_low,win_y_low),
        (win_xleft_high,win_y_high),(0,255,0), 2) 
        cv2.rectangle(out_img,(win_xright_low,win_y_low),
        (win_xright_high,win_y_high),(0,255,0), 2) 
        
        # Identify the nonzero pixels in x and y within the window #
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
        
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices (previously was a list of lists of pixels)
    try:
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)
    except ValueError:
        # Avoids an error if the above is not implemented fully
        pass

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    return leftx, lefty, rightx, righty, out_img


def fit_poly(img_shape, leftx, lefty, rightx, righty):
     ### TO-DO: Fit a second order polynomial to each with np.polyfit() ###
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    # Generate x and y values for plotting
    ploty = np.linspace(0, img_shape[0]-1, img_shape[0])
    ### TO-DO: Calc both polynomials using ploty, left_fit and right_fit ###
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    
    return left_fit, right_fit, left_fitx, right_fitx, ploty


def fit_polynomial(binary_warped):
    """finding lane line pixels using the function 'find_lane_pixels'"""
    """fit a polynom to the found lane line pixels"""
    
    # Define conversions in x and y from pixels space to meters
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel in x dimension
    
    # Find our lane pixels first
    leftx, lefty, rightx, righty, out_img = find_lane_pixels(binary_warped)

    # Fit a second order polynomial to each using `np.polyfit` in pixel space
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    
    # Fit a second order polynomial to each using `np.polyfit` in meter space
    left_fit_real = np.polyfit(lefty*ym_per_pix, leftx*xm_per_pix, 2)
    right_fit_real = np.polyfit(righty*ym_per_pix, rightx*xm_per_pix, 2)

    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    try:
        left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
        right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    except TypeError:
        # Avoids an error if `left` and `right_fit` are still none or incorrect
        print('The function failed to fit a line!')
        left_fitx = 1*ploty**2 + 1*ploty
        right_fitx = 1*ploty**2 + 1*ploty

    ## Visualization ##
    # Colors in the left and right lane regions
    out_img[lefty, leftx] = [255, 0, 0]
    out_img[righty, rightx] = [0, 0, 255]

    # Plots the left and right polynomials on the lane lines
    #plt.plot(left_fitx, ploty, color='yellow')
    #plt.plot(right_fitx, ploty, color='yellow')

    return out_img, left_fit, right_fit, left_fit_real, right_fit_real, left_fitx, right_fitx

def search_around_poly(binary_warped, left_fit, right_fit):
    # HYPERPARAMETER
    # Choose the width of the margin around the previous polynomial to search
    # The quiz grader expects 100 here, but feel free to tune on your own!
    margin = 100

    # Grab activated pixels
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    
    ### TO-DO: Set the area of search based on activated x-values ###
    ### within the +/- margin of our polynomial function ###
    ### Hint: consider the window areas for the similarly named variables ###
    ### in the previous quiz, but change the windows to our new search area ###
    left_lane_inds = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + 
                    left_fit[2] - margin)) & (nonzerox < (left_fit[0]*(nonzeroy**2) + 
                    left_fit[1]*nonzeroy + left_fit[2] + margin)))
    right_lane_inds = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + 
                    right_fit[2] - margin)) & (nonzerox < (right_fit[0]*(nonzeroy**2) + 
                    right_fit[1]*nonzeroy + right_fit[2] + margin)))
    
    # Again, extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    # Fit new polynomials
    left_fit, right_fit, left_fitx, right_fitx, ploty = fit_poly(binary_warped.shape, leftx, lefty, rightx, righty)
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel in x dimension
    left_fit_real = np.polyfit(lefty*ym_per_pix, leftx*xm_per_pix, 2)
    right_fit_real = np.polyfit(righty*ym_per_pix, rightx*xm_per_pix, 2)
    
    ## Visualization ##
    # Create an image to draw on and an image to show the selection window
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
    window_img = np.zeros_like(out_img)
    # Color in left and right line pixels
    out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
    out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]

    # Generate a polygon to illustrate the search window area
    # And recast the x and y points into usable format for cv2.fillPoly()
    left_line_window1 = np.array([np.transpose(np.vstack([left_fitx-margin, ploty]))])
    left_line_window2 = np.array([np.flipud(np.transpose(np.vstack([left_fitx+margin, 
                              ploty])))])
    left_line_pts = np.hstack((left_line_window1, left_line_window2))
    right_line_window1 = np.array([np.transpose(np.vstack([right_fitx-margin, ploty]))])
    right_line_window2 = np.array([np.flipud(np.transpose(np.vstack([right_fitx+margin, 
                              ploty])))])
    right_line_pts = np.hstack((right_line_window1, right_line_window2))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(window_img, np.int_([left_line_pts]), (0,255, 0))
    cv2.fillPoly(window_img, np.int_([right_line_pts]), (0,255, 0))
    result = cv2.addWeighted(out_img, 1, window_img, 0.3, 0)
    
    # Plot the polynomial lines onto the image
    #plt.plot(left_fitx, ploty, color='yellow')
    #plt.plot(right_fitx, ploty, color='yellow')
    ## End visualization steps ##
    
    return result, left_fit, right_fit, left_fit_real, right_fit_real, left_fitx, right_fitx, ploty

def save_lane_data(laneData, fitx, ploty):
    
    # save x values of current fit
    laneData.recent_xfitted = np.vstack((fitx,laneData.recent_xfitted))
    
    # average x values of nFrame_avg frames
    s = np.ma.size(laneData.recent_xfitted,1)
    if(s<nFrame_avg):
       laneData.bestx = np.mean(laneData.recent_xfitted, axis=0)
    else:
        laneData.bestx = np.mean(laneData.recent_xfitted[0:nFrame_avg,:], axis=0)
    
    # refit polynomial coeff
    laneData.best_fit = np.polyfit(ploty, laneData.bestx, 2)
    laneData.best_fit_real = np.polyfit(ploty*ym_per_pix, laneData.bestx*xm_per_pix, 2)
    
    # calculate curvarture based on best fit
    laneData.radius_of_curvature = measure_curvature_real(ploty, laneData.best_fit_real)
    
    return laneData

def check_detection(laneLeft, laneRight, left_fit, right_fit, left_fit_real, right_fit_real, ploty):
    
    # initiate boolean
    right_detected = True
    left_detected = True
    
    ## check radius difference
    # calculate difference in Radius between both lanes
    left_curvrad = measure_curvature_real( ploty, left_fit_real)
    right_curvrad = measure_curvature_real( ploty, right_fit_real)
    diff_L_R = abs(left_curvrad-right_curvrad)
    
    # calculate difference to prior detected radius
    diff_to_prior_left = abs(laneLeft.radius_of_curvature-left_curvrad)
    diff_to_prior_right = abs(laneRight.radius_of_curvature-right_curvrad)
    
    # calculate slope
    yslope = np.mean(ploty)
    slope_L = 2*left_fit_real[0]*yslope+left_fit_real[1]
    slope_R = 2*right_fit_real[0]*yslope+right_fit_real[1]
    diff_slope = abs(slope_R-slope_L)
    
    if(diff_L_R>max_diff_rad_L_R):
        
        if(diff_to_prior_left>max_diff_to_prior): 
            left_detected=False
           
        if(diff_to_prior_right>max_diff_to_prior):
            right_detected=False
            
    return left_detected, right_detected, diff_L_R, diff_to_prior_left, diff_to_prior_right, slope_L, slope_R, diff_slope 

## classes

# Define a class to receive the characteristics of each line detection
class Line():
    def __init__(self):
        # was the line detected in the last iteration?
        self.detected = False  
        # number of frames without detection
        self.n_non_detected = 0 
        # x values of the last n fits of the line
        self.recent_xfitted = []
        #average x values of the fitted line over the last n iterations
        self.bestx = None     
        #polynomial coefficients averaged over the last n iterations
        self.best_fit = [np.array([0,0,0])] 
        self.best_fit_real = [np.array([0,0,0])] 
        #polynomial coefficients for the most recent fit
        self.current_fit = [np.array([0,0,0])]  
        self.current_fit_real = [np.array([0,0,0])]  
        #radius of curvature of the line in some units
        self.radius_of_curvature = None 
        #distance in meters of vehicle center from the line
        self.line_base_pos = None 
        #difference in fit coefficients between last and new fits
        self.diffs = np.array([0,0,0], dtype='float') 
        #x values for detected line pixels
        self.allx = None  
        #y values for detected line pixels
        self.ally = None  

## Calculate curvature and displacement of the car relativ to the lane-centre

In [25]:
# helper functions
import numpy as np
    
def measure_curvature_pixels(ploty, fit):
    '''
    Calculates the curvature of polynomial functions in pixels.
    '''
    # Define y-value where we want radius of curvature
    # We'll choose the maximum y-value, corresponding to the bottom of the image
    y_eval = np.max(ploty)
    
    # Calculation of R_curve (radius of curvature)
    curverad = ((1 + (2*fit[0]*y_eval + fit[1])**2)**1.5) / np.absolute(2*fit[0])
    
    return curverad

def measure_curvature_real(ploty, fit_real):
    '''
    Calculates the curvature of polynomial functions in meters.
    '''
    
    # Start by generating our fake example data
    # Make sure to feed in your real data instead in your project!
    
    # Define y-value where we want radius of curvature
    # We'll choose the maximum y-value, corresponding to the bottom of the image
    y_eval = np.max(ploty)
    
    # Calculation of R_curve (radius of curvature)
    curverad = ((1 + (2*fit_real[0]*y_eval*ym_per_pix + fit_real[1])**2)**1.5) / np.absolute(2*fit_real[0])
    
    return curverad

def measure_displacement_from_center(img, xm_per_pix, right_fitx, left_fitx):
    
    # calculate center of image
    pos_center_img = (img.shape[1]/2)*xm_per_pix
    
    # calculate space between both detected lines
    width_lane = ((right_fitx[-1] - left_fitx[-1]))*xm_per_pix
    
    #calculate center lane line
    pos_center_lane = left_fitx[-1]*xm_per_pix + width_lane/2
    
    # calculate displcement
    displacement = (pos_center_lane - pos_center_img)
    
    return displacement

## Visualization

In [8]:
def show_lane_line(binary_warped, image, left_fitx, right_fitx, ploty, Minv):
    # Create an image to draw the lines on
    warp_zero = np.zeros_like(binary_warped).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))

    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, Minv, (image.shape[1], image.shape[0])) 
    # Combine the result with the original image
    result = cv2.addWeighted(image, 1, newwarp, 0.3, 0)
    return result

def print_data(img, rad_left, rad_right, diff_L_R, diff_to_prior_left, diff_to_prior_right, slope_L, slope_R, diff_slope, displacement):
    
    font = cv2.FONT_HERSHEY_SIMPLEX
    #cv2.putText(img,'radius_left = '+ repr(rad_left),(10,100), font, 1,(255,255,255),2,cv2.LINE_AA)
    #cv2.putText(img,'radius_right = '+ repr(rad_right),(10,150), font, 1,(255,255,255),2,cv2.LINE_AA)
    #cv2.putText(img,'rad_diff_L_R = '+ repr(diff_L_R),(10,200), font, 1,(255,255,255),2,cv2.LINE_AA)
    #cv2.putText(img,'rad_diff_to_prior_L = '+ repr(diff_to_prior_left),(10,250), font, 1,(255,255,255),2,cv2.LINE_AA)
    #cv2.putText(img,'rad_diff_to_prior_R = '+ repr(diff_to_prior_right),(10,300), font, 1,(255,255,255),2,cv2.LINE_AA)
    #cv2.putText(img,'slope_L = '+ repr(slope_L),(10,350), font, 1,(255,255,255),2,cv2.LINE_AA)
    #cv2.putText(img,'slope_R = '+ repr(slope_R),(10,400), font, 1,(255,255,255),2,cv2.LINE_AA)
    #cv2.putText(img,'diff_slope = '+ repr(diff_slope),(10,450), font, 1,(255,255,255),2,cv2.LINE_AA)
    cv2.putText(img,'Radius = '+ repr(float("{0:.2f}".format((rad_left+rad_right)/2)))+"m",(10,100), font, 1,(255,255,255),2,cv2.LINE_AA)
    cv2.putText(img,'Displacement = '+ repr(float("{0:.2f}".format((displacement)/2)))+"m",(10,150), font, 1,(255,255,255),2,cv2.LINE_AA)
    
    return img



## Advanced Lane Finding Test - Pipeline

In [9]:
## Initilization
ym_per_pix = 30/720 # meters per pixel in y dimension
xm_per_pix = 3.7/700 # meters per pixel in x dimension

# set camera calibration parameters
nx = 9 # number chessboard corners in x direction
ny = 6 # number chessboard corners in y direction

# camera calibration
# load image
img = mpimg.imread('./test_images/test6.jpg')
objpoints, imgpoints = search_corners(nx,ny)
mtx, dist = cal_undistort(img.shape, objpoints, imgpoints)

# set parameters for transformation into thresholded binary image
sobel_thresh_min = 20  # threshold sobel x
sobel_thresh_max = 100
s_thresh_min = 90    # threshold s channel for HLS color space
s_thresh_max = 255

# set parameter for mask (region of interest) and reference point for bird view
# Define vertices of mask
dx_r1 = 0 #point 1 - bottom left (distance from rigth corner)
dx_l1 = 0 #point 2 - bottom right (distance from left corner)
dx_r2 = 100 #point 3 - top left (distance from center)
dx_l2 = dx_r2 #point 4 - top right (distance from center)
dy = 450

# set parameter for averaging process
maxDiff = 100 # of Radius

# camera calibration
objpoints, imgpoints = search_corners(nx,ny)

# initiate line class
laneLeft = Line()
laneRight = Line()

# frame count
nFrame = 0
nFrame_avg = 10 #frames 

# thresholds for checking detection
max_diff_rad_L_R = 50; #meter
max_diff_to_prior = 50; #difference in radius
max_non_detected = 60;

In [24]:
def advanced_lane_line_finding(img):

    # increase frame number
    global nFrame
    global laneLeft
    global laneRight
    nFrame +=1
    
    # init check detection variables
    diff_L_R = 0
    diff_to_prior_left = 0
    diff_to_prior_right = 0
    slope_L = 0
    slope_R = 0
    diff_slope = 0
    
    # undistor image
    undist = undist_img(img, mtx, dist)

    # color transforms & gradients to create a thresholded binary image
    color_binary, combined_binary = convert_to_bin(undist,sobel_thresh_min, sobel_thresh_max, s_thresh_min, s_thresh_max)

    # create masked binary image
    imshape = img.shape
    mask_vertices = np.array([[(0+dx_l1,imshape[0]),(imshape[1]/2 - dx_l2, dy), (imshape[1]/2 + dx_r2, dy), (imshape[1]-dx_r1,imshape[0])]], dtype=np.int32)
    masked_binary = region_of_interest(combined_binary, mask_vertices)
    
    # transform image to birdsview image
    xmin = mask_vertices[0,0,0] + 200
    xmax = mask_vertices[0,3,0] - 200
    ymin = 0
    ymax = imshape[0]
    ratioWarp = ((xmax-xmin)/ (mask_vertices[0,3,0]-mask_vertices[0,0,0]))
    src = np.float32([[mask_vertices[0,0,0:2]],[mask_vertices[0,1,0:2]],[mask_vertices[0,2,0:2]],[mask_vertices[0,3,0:2]]])
    dst = np.float32([[xmin,ymax],[xmin-250,ymin],[xmax+250,ymin],[xmax,ymax]])
    bird_view, M = convert_to_birdview(masked_binary, src, dst)
    
    # lane detection for the first nFrame_avg frames
    if(nFrame<=nFrame_avg):
        # find lane line pixels and fit a 2nd order polynom to the found lane line pixels
        result, left_fit, right_fit, left_fit_real, right_fit_real, left_fitx, right_fitx = fit_polynomial(bird_view)
        
        if(nFrame == 1):
            laneLeft.recent_xfitted = left_fitx
            laneRight.recent_xfitted = right_fitx
        
        # safe lane data
        ploty = np.linspace(0, img.shape[0]-1, img.shape[0])
        laneLeft = save_lane_data(laneLeft, left_fitx, ploty)
        laneRight = save_lane_data(laneRight, right_fitx, ploty)
        
        
        # check detection
        laneLeft.detected, laneRight.detected, diff_L_R, diff_to_prior_left, diff_to_prior_right, slope_L, slope_R, diff_slope\
        = check_detection(laneLeft, laneRight, left_fit, right_fit, left_fit_real, right_fit_real, ploty)
        
        # set detection flag
        laneLeft.detected = True
        laneRight.detected = True
        laneLeft.n_non_detected = 0
        laneRight.n_non_detected = 0
        
    elif((nFrame>nFrame_avg)and(laneLeft.n_non_detected<max_non_detected) and \
         (laneRight.n_non_detected<max_non_detected)):
        # search for lane line pixels around a given polynom and re-fit 2nd order polynom
        result, left_fit, right_fit, left_fit_real, right_fit_real, left_fitx, right_fitx, ploty \
        = search_around_poly(bird_view, laneLeft.best_fit, laneRight.best_fit)
        
        # check detection
        laneLeft.detected, laneRight.detected, diff_L_R, diff_to_prior_left, diff_to_prior_right, slope_L, slope_R, diff_slope \
        = check_detection(laneLeft, laneRight, left_fit, right_fit, left_fit_real, right_fit_real, ploty)

        laneLeft = save_lane_data(laneLeft, left_fitx, ploty)
        laneLeft.n_non_detected = 0

        laneRight = save_lane_data(laneRight, right_fitx, ploty)
        laneRight.n_non_detected = 0
        
    elif(laneLeft.n_non_detected>max_non_detected) or (laneRight.n_non_detected>max_non_detected):
         
        # find lane line pixels and fit a 2nd order polynom to the found lane line pixels
        result, left_fit, right_fit, left_fit_real, right_fit_real, left_fitx, right_fitx = fit_polynomial(bird_view)

        # save lane data if detected
        ploty = np.linspace(0, img.shape[0]-1, img.shape[0])
        laneLeft = save_lane_data(laneLeft, left_fitx, ploty)
        laneLeft.n_non_detected = 0
            
        # save lane data if detected
        laneRight = save_lane_data(laneRight, right_fitx, ploty)
        laneRight.n_non_detected = 0
        
        #print("part reset")
        
    # calculate displacement
    displacement = measure_displacement_from_center(img, xm_per_pix*ratioWarp, right_fitx, left_fitx)
    
    # visualize result
    Minv = np.linalg.inv(M)
    ploty = np.linspace(0, img.shape[0]-1, img.shape[0])
    left_fitx = laneLeft.best_fit[0]*ploty**2 + laneLeft.best_fit[1]*ploty + laneLeft.best_fit[2]
    right_fitx = laneRight.best_fit[0]*ploty**2 + laneRight.best_fit[1]*ploty + laneRight.best_fit[2]
    
    
    result = show_lane_line(bird_view, undist, left_fitx, right_fitx, ploty, Minv)
    result = print_data(result, laneLeft.radius_of_curvature, laneRight.radius_of_curvature, \
                        diff_L_R, diff_to_prior_left, diff_to_prior_right, slope_L, slope_R, diff_slope, displacement)
    
    #return result, undist, color_binary, combined_binary, masked_binary, bird_view, left_fit, right_fit, ploty
    return result
    
    

## Run pipeline on all test images

In [ ]:
import os

# read files in directory
images = os.listdir("test_images/")

for i in images:
    
    # reseting variables
    nFrame = 0

    # initiate line class
    laneLeft = Line()
    laneRight = Line()
    
    # load image
    j = 'test_images/' + i
    img = mpimg.imread(j)
    
    # Run pipeline on the test images
    result, undist, color_binary, combined_binary, masked_binary, bird_view, left_fit, right_fit, ploty = advanced_lane_line_finding(img)

    # save image
    mpimg.imsave("output_images/"+i,result)

## Test pipeline with video

In [26]:
# reseting variables
nFrame = 0

# initiate line class
laneLeft = Line()
laneRight = Line()

white_output = 'test_videos_output/project_video.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(0,5)
clip1 = VideoFileClip("project_video.mp4")
white_clip = clip1.fl_image(advanced_lane_line_finding) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

[MoviePy] >>>> Building video test_videos_output/project_video.mp4
[MoviePy] Writing video test_videos_output/project_video.mp4



 12%|█▏        | 152/1261 [00:36<04:39,  3.97it/s]


 24%|██▍       | 303/1261 [01:14<04:38,  3.43it/s]


 36%|███▌      | 454/1261 [01:53<03:19,  4.04it/s]


 48%|████▊     | 605/1261 [02:33<03:02,  3.59it/s]


...

 72%|███████▏  | 907/1261 [03:52<01:37,  3.64it/s]


 84%|████████▍ | 1057/1261 [04:34<01:01,  3.31it/s]


 96%|█████████▌| 1205/1261 [05:15<00:15,  3.69it/s]


100%|█████████▉| 1260/1261 [05:30<00:00,  3.86it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/project_video.mp4 

CPU times: user 3min 17s, sys: 2.41 s, total: 3min 19s
Wall time: 5min 33s


In [27]:
HTML("""
<video width="960" height="
540" controls>
  <source src="{0}">
</video>
""".format(white_output))

## Generate plots 


In [ ]:
# load image
img = mpimg.imread('./test_images/test6.jpg')
    
# Run pipeline on the test images
result, undist, color_binary, combined_binary, masked_binary, bird_view, left_fit, right_fit, ploty = advanced_lane_line_finding(img)

# show undistorted image
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
f.tight_layout()
ax1.imshow(img)
ax1.set_title('Original Image', fontsize=50)
ax2.imshow(undist)
ax2.set_title('Undistorted Image', fontsize=50)
plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
plt.show()
plt.savefig('./write_up_images/undist.jpg')
    
# show binary images
# Plotting thresholded images
f, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(20,10))
ax1.set_title('Undistorted Image')
ax1.imshow(undist)
ax2.set_title('Binary image')
ax2.imshow(combined_binary, cmap='gray')
ax3.set_title('Masked binary image')
ax3.imshow(masked_binary, cmap='gray')
plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
plt.show()
plt.savefig('./write_up_images/binary.jpg')

# Plotting bird view 
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
ax1.set_title('Masked binary image')
ax1.imshow(masked_binary, cmap='gray')
ax2.set_title('Bird view of binary image')
ax2.imshow(bird_view, cmap='gray')
plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
plt.show()
plt.savefig('./write_up_images/binary_bird.jpg')

# Plottting polynomials
def fit_poly2(img_shape, leftx, lefty, rightx, righty):
     ### TO-DO: Fit a second order polynomial to each with np.polyfit() ###
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    # Generate x and y values for plotting
    ploty = np.linspace(0, img_shape[0]-1, img_shape[0])
    ### TO-DO: Calc both polynomials using ploty, left_fit and right_fit ###
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    
    return left_fitx, right_fitx, ploty

def search_around_poly2(binary_warped):
    # HYPERPARAMETER
    # Choose the width of the margin around the previous polynomial to search
    # The quiz grader expects 100 here, but feel free to tune on your own!
    margin = 4

    # Grab activated pixels
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    
    ### TO-DO: Set the area of search based on activated x-values ###
    ### within the +/- margin of our polynomial function ###
    ### Hint: consider the window areas for the similarly named variables ###
    ### in the previous quiz, but change the windows to our new search area ###
    left_lane_inds = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + 
                    left_fit[2] - margin)) & (nonzerox < (left_fit[0]*(nonzeroy**2) + 
                    left_fit[1]*nonzeroy + left_fit[2] + margin)))
    right_lane_inds = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + 
                    right_fit[2] - margin)) & (nonzerox < (right_fit[0]*(nonzeroy**2) + 
                    right_fit[1]*nonzeroy + right_fit[2] + margin)))
    
    # Again, extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    # Fit new polynomials
    left_fitx, right_fitx, ploty = fit_poly2(binary_warped.shape, leftx, lefty, rightx, righty)
    
    ## Visualization ##
    # Create an image to draw on and an image to show the selection window
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
    window_img = np.zeros_like(out_img)
    # Color in left and right line pixels
    out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
    out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]

    # Generate a polygon to illustrate the search window area
    # And recast the x and y points into usable format for cv2.fillPoly()
    left_line_window1 = np.array([np.transpose(np.vstack([left_fitx-margin, ploty]))])
    left_line_window2 = np.array([np.flipud(np.transpose(np.vstack([left_fitx+margin, 
                              ploty])))])
    left_line_pts = np.hstack((left_line_window1, left_line_window2))
    right_line_window1 = np.array([np.transpose(np.vstack([right_fitx-margin, ploty]))])
    right_line_window2 = np.array([np.flipud(np.transpose(np.vstack([right_fitx+margin, 
                              ploty])))])
    right_line_pts = np.hstack((right_line_window1, right_line_window2))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(window_img, np.int_([left_line_pts]), (0,255, 0))
    cv2.fillPoly(window_img, np.int_([right_line_pts]), (0,255, 0))
    result = cv2.addWeighted(out_img, 1, window_img, 1, 0)
    
    # Plot the polynomial lines onto the image
    plt.plot(left_fitx, ploty, color='yellow')
    plt.plot(right_fitx, ploty, color='yellow')
    ## End visualization steps ##
    
    return result

binary_warped = bird_view
img_poly = search_around_poly2(bird_view)
f, (ax1) = plt.subplots(1, 1, figsize=(20,10))
ax1.set_title('Detected lane lines')
ax1.imshow(img_poly )
plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
plt.show()
plt.savefig('./write_up_images/detected_lines.jpg')    

In [ ]:
# plot distorted and undistorted image
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
f.tight_layout()
ax1.imshow(img[:,:,[2,1,0]])
ax1.set_title('Original Image', fontsize=50)
ax2.imshow(undist[:,:,[2,1,0]])
ax2.set_title('Undistorted Image', fontsize=50)
plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

In [ ]:
# Plotting thresholded images
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
ax1.set_title('Stacked thresholds')
ax1.imshow(color_binary)
ax2.set_title('Combined S channel and gradient thresholds')
ax2.imshow(combined_binary, cmap='gray')
plt.show()

In [ ]:
 # Plotting masked thresholded images
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
ax1.set_title('Combined S channel and gradient thresholds')
ax1.imshow(combined_binary)
ax2.set_title('Masked & ombined S channel and gradient thresholds')
ax2.imshow(masked_binary, cmap='gray')
plt.show()

In [ ]:
 # Plotting bird view thresholded images
cv2.line(bird_view, (xmin+200, ymin), (xmin+200, ymax), color=[255, 0, 0], thickness=2)
cv2.line(bird_view, (xmax-200, ymin), (xmax-200, ymax), color=[255, 0, 0], thickness=2)
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
ax1.set_title('Combined S channel and gradient thresholds')
ax1.imshow(masked_binary)
ax2.set_title('Masked & ombined S channel and gradient thresholds')
ax2.imshow(bird_view, cmap='gray')
plt.show()

In [ ]:
# plotting histogram
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
ax1.set_title('Combined S channel and gradient thresholds')
ax1.imshow(bird_view, cmap='gray')
ax2.set_title('Masked & ombined S channel and gradient thresholds')
ax2.plot(histogram)
plt.show()


In [ ]:
# plotting polynom
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
ax1.set_title('Combined S channel and gradient thresholds')
ax1.imshow(bird_view, cmap='gray')
ax2.set_title('Masked & ombined S channel and gradient thresholds')
ax2.imshow(out_img)
plt.show()

In [ ]:
# plotting polynom
print(left_curverad_real)
print(right_curverad_real)
plt.imshow(out_img)
plt.show()

In [ ]:
img = mpimg.imread('./test_images/straight_lines1.jpg')
#img = mpimg.imread('./test_images/test6.jpg')
imshape = img.shape
mask_vertices = np.array([[(0+dx_l1,imshape[0]),(imshape[1]/2 - dx_l2, dy), (imshape[1]/2 + dx_r2, dy), (imshape[1]-dx_r1,imshape[0])]], dtype=np.int32)
xmin = mask_vertices[0,0,0] + 200
xmax = mask_vertices[0,3,0] - 200
ymin = 0
ymax = imshape[0]
src = np.float32([[mask_vertices[0,0,0:2]],[mask_vertices[0,1,0:2]],[mask_vertices[0,2,0:2]],[mask_vertices[0,3,0:2]]])
dst = np.float32([[xmin,ymax],[xmin-250,ymin],[xmax+250,ymin],[xmax,ymax]])
bird_view, M = convert_to_birdview(img, src, dst)
 # Plotting bird view thresholded images
cv2.line(bird_view, (xmin+200, ymin), (xmin+200, ymax), color=[255, 0, 0], thickness=2)
cv2.line(bird_view, (xmax-200, ymin), (xmax-200, ymax), color=[255, 0, 0], thickness=2)
plt.imshow(bird_view, cmap='gray')
plt.show()
